## Notebook for Applied Data Science Capstone project 

In [10]:
import numpy as np
import pandas as pd

print("Hello Capstone Project Course!")

Hello Capstone Project Course!


## Peer graded assignment Week 3 - Segmenting and clustering Toronto into neighborhoods 

### 1. Retrieve and parse the Wikipedia page and create the Data Frame ###

**Downloading and importing all the necessary pacakges and libraries**

In [11]:
!pip install beautifulsoup4
!pip install requests
!pip install lxml

from bs4 import BeautifulSoup
import requests
import pandas as pd

**Getting the wikipedia page**

In [12]:
#using requests to get the page
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
#get the html in a soup object
soup = BeautifulSoup(source, 'lxml')

**Parsing the soup object to get the columns and create the df**

In [13]:
#isolating the table from the HTML
table = soup.find('tbody')

columns = []
#parsing the table to get the columns and create the df
for column in table.find_all('th'):
    columns.append(column.text.replace("\n",""))

df = pd.DataFrame(columns=columns)
df

,Postal code,Borough,Neighborhood


**Parsing the remainder of the html table and populate the df**

In [14]:
for row in table.find_all('tr'):
    row_splitted = []
    #check that the row values are consistent and the row is not empty or the header
    if (str(row).count('Not assigned') == 0) and (row.find_all('td')):
        for data in row.find_all('td'):
            #format the string to remove unwanted characters
            row_splitted.append(data.text.replace("\n","").replace(' / ', ', '))
        #insert the row in the df    
        row_series = pd.Series(row_splitted, index = df.columns)
        df = df.append(row_series, ignore_index=True)

df

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [15]:
df.shape

(103, 3)

### 2. Retrieve and add the geospatial informations ###

**Download the geospatial data of the neighborhoods into a Pandas df**

In [16]:
file_name = 'http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv'
df_lonlat = pd.read_csv(file_name)
df_lonlat.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


**Joining dataframes on the 'Postal code'**

In [17]:
#Join the dataframes on the postal code
df_join = df.join(df_lonlat.set_index('Postal Code'), on='Postal code')
df_join.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


### 3. Clustering the neighborhoods and plot them on a map ### 

In [18]:
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    openssl-1.1.1f             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                       

**Get some basic info on the datas**

In [19]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_join['Borough'].unique()),
        df_join.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


In [20]:
df_join.describe()

,Latitude,Longitude
count,103.000000,103.000000
mean,43.704608,-79.397153
std,0.052463,0.097146
min,43.602414,-79.615819
25%,43.660567,-79.464763
50%,43.696948,-79.388790
75%,43.745320,-79.340923
max,43.836125,-79.160497


**To center the map i can use the mean of the Latitude and longitude**

Is not an optimal solution as areas with higher neighborhood density will have more weight but is a better alternative then picking a random neighborhood and as an approximation for this exercise should be enough.

In [21]:
latitude = df_join.describe()['Latitude']['mean']
longitude = df_join.describe()['Longitude']['mean']

**Vizualize the neighborhoods**

In [22]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_join['Latitude'], df_join['Longitude'], df_join['Borough'], df_join['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

**Reducing the dataframe to the Boroughs that contain 'Toronto' in the name**

In [23]:
df_toronto = df_join[df_join['Borough'].str.contains('Toronto')].reset_index(drop=True)
df_toronto.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


**Exploring the data set**

In [24]:
df_toronto.shape

(39, 5)

In [25]:
df_toronto.describe()

,Latitude,Longitude
count,39.000000,39.000000
mean,43.667135,-79.389873
std,0.023478,0.037451
min,43.628947,-79.484450
25%,43.649765,-79.405678
50%,43.662301,-79.387383
75%,43.677957,-79.376474
max,43.728020,-79.293031


In [26]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_toronto['Borough'].unique()),
        df_toronto.shape[0]
    )
)

The dataframe has 4 boroughs and 39 neighborhoods.


Once again i'll use the mean latitude and longitude to center the map, this time should be more accurate as we are focusing on the central area and the distribution of neighborhoods should be more uniform 

In [27]:
latitude = df_toronto.describe()['Latitude']['mean']
longitude = df_toronto.describe()['Longitude']['mean']

**Visualizing the new map**

In [28]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Adding Foursquare credentials

In [ ]:
CLIENT_ID = 'your Foursquare ID' # your Foursquare ID
CLIENT_SECRET = 'your Foursquare Secret' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

**Retrieve the 100 most popular venues in each neighborhood according to Foursquare**

In [30]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [31]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [32]:
toronto_venues = getNearbyVenues(names=df_toronto['Neighborhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town,

**toronto_venue now contains all the most popular venues along with their category and geospatial info**

In [33]:
print(toronto_venues.shape)
toronto_venues.head()

(1683, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


**Let's have a quick look at how many venues Foursquare returned for each neighborhood**

In [34]:
toronto_venues['Neighborhood'].value_counts()

Harbourfront East, Union Station, Toronto Islands                                                             100
St. James Town                                                                                                100
Richmond, Adelaide, King                                                                                      100
First Canadian Place, Underground city                                                                        100
Commerce Court, Victoria Hotel                                                                                100
Garden District, Ryerson                                                                                      100
Toronto Dominion Centre, Design Exchange                                                                      100
Stn A PO Boxes                                                                                                 97
Church and Wellesley                                                                    

In [35]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 233 uniques categories.


**Modifying the df to change the categorical value of 'Venue Category' in a numerical one**

This process is needed to better evaluate the frequency of each venue category in each neighborhood.
Will use a one hot encoding to do this transformation.

In [47]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
print(toronto_onehot.shape)
# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head() 


(1683, 233)


,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Toy / Game Store,Trail,Train Station,Transportation Service,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [69]:
toronto_onehot.shape

(1683, 233)

**Get the mean for each venue category in each area**

In [48]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Transportation Service,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,Berczy Park,0.000000,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.017544,0.000000,0.000000,0.000000,0.000000,0.00
1,"Brockton, Parkdale Village, Exhibition Place",0.043478,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
2,Business reply mail Processing CentrE,0.055556,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.000000,0.0625,0.0625,0.125,0.1875,0.125,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
4,Central Bay Street,0.013158,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.013158,0.000000,...,0.000000,0.00000,0.00,0.000000,0.013158,0.000000,0.000000,0.013158,0.000000,0.00
5,Christie,0.000000,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
6,Church and Wellesley,0.025316,0.012658,0.0000,0.0000,0.000,0.0000,0.000,0.012658,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.012658,0.00
7,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.040000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.020000,0.000000,0.000000,0.010000,0.000000,0.00
8,Davisville,0.000000,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.026316,0.000000,...,0.026316,0.00000,0.00,0.026316,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
9,Davisville North,0.000000,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00


In [49]:
toronto_grouped.shape

(39, 233)

Let's get the top 10 most popular venues for each neighborhood just to better understand the data 

In [50]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [51]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Cheese Shop,Café,Restaurant,Seafood Restaurant,Beer Bar,Farmers Market,Greek Restaurant
1,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Coffee Shop,Grocery Store,Gym,Pet Store,Performing Arts Venue,Nightclub,Italian Restaurant,Intersection
2,Business reply mail Processing CentrE,Light Rail Station,Yoga Studio,Recording Studio,Restaurant,Burrito Place,Fast Food Restaurant,Auto Workshop,Farmers Market,Spa,Smoke Shop
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Boat or Ferry,Harbor / Marina,Sculpture Garden,Rental Car Location,Plane,Bar,Boutique
4,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Japanese Restaurant,Sandwich Place,Salad Place,Department Store,Bubble Tea Shop,Thai Restaurant,Dessert Shop


**Cluster Neighborhoods**

We feed the K-Means alghorithm the dataframe with the frequencies of each category without the neighborhood name

In [93]:
toronto_grouped_clustering

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Toy / Game Store,Trail,Train Station,Transportation Service,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,0.000000,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.00,...,0.000000,0.00000,0.00,0.000000,0.017544,0.000000,0.000000,0.000000,0.000000,0.00
1,0.043478,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.00,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
2,0.055556,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.00,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
3,0.000000,0.000000,0.0625,0.0625,0.125,0.1875,0.125,0.000000,0.000000,0.00,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
4,0.013158,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.013158,0.000000,0.00,...,0.000000,0.00000,0.00,0.000000,0.013158,0.000000,0.000000,0.013158,0.000000,0.00
5,0.000000,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.00,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
6,0.025316,0.012658,0.0000,0.0000,0.000,0.0000,0.000,0.012658,0.000000,0.00,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.012658,0.00
7,0.000000,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.040000,0.000000,0.00,...,0.000000,0.00000,0.00,0.000000,0.020000,0.000000,0.000000,0.010000,0.000000,0.00
8,0.000000,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.026316,0.000000,0.00,...,0.026316,0.00000,0.00,0.026316,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
9,0.000000,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.00,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00


In [53]:
# import k-means from clustering stage
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

Merging the df to get cluster labels as well as common venues and neighborhood datas

In [54]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Park,Bakery,Pub,Mexican Restaurant,Breakfast Spot,Restaurant,Café,Theater,Electronics Store
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Diner,Bank,Beer Bar,Boutique,Sandwich Place,Burger Joint,Burrito Place,Café,Park
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Clothing Store,Coffee Shop,Café,Middle Eastern Restaurant,Japanese Restaurant,Bubble Tea Shop,Cosmetics Shop,Tea Room,Electronics Store,Bookstore
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Café,Cocktail Bar,Restaurant,Italian Restaurant,Clothing Store,Beer Bar,Diner,American Restaurant,Bakery
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Trail,Pub,Women's Store,Department Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


Now that we have the dataframe clustered we proceed to visualize the result

In [56]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

We can see that Toronto appears very homogeneous let's analyze why considering the first (and most common) cluster

In [60]:
cluster1 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]
cluster1

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Regent Park, Harbourfront",0,Coffee Shop,Park,Bakery,Pub,Mexican Restaurant,Breakfast Spot,Restaurant,Café,Theater,Electronics Store
1,"Queen's Park, Ontario Provincial Government",0,Coffee Shop,Diner,Bank,Beer Bar,Boutique,Sandwich Place,Burger Joint,Burrito Place,Café,Park
2,"Garden District, Ryerson",0,Clothing Store,Coffee Shop,Café,Middle Eastern Restaurant,Japanese Restaurant,Bubble Tea Shop,Cosmetics Shop,Tea Room,Electronics Store,Bookstore
3,St. James Town,0,Coffee Shop,Café,Cocktail Bar,Restaurant,Italian Restaurant,Clothing Store,Beer Bar,Diner,American Restaurant,Bakery
4,The Beaches,0,Health Food Store,Trail,Pub,Women's Store,Department Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
5,Berczy Park,0,Coffee Shop,Cocktail Bar,Bakery,Cheese Shop,Café,Restaurant,Seafood Restaurant,Beer Bar,Farmers Market,Greek Restaurant
6,Central Bay Street,0,Coffee Shop,Café,Italian Restaurant,Japanese Restaurant,Sandwich Place,Salad Place,Department Store,Bubble Tea Shop,Thai Restaurant,Dessert Shop
7,Christie,0,Grocery Store,Café,Park,Nightclub,Coffee Shop,Italian Restaurant,Baby Store,Restaurant,Diner,Candy Store
8,"Richmond, Adelaide, King",0,Coffee Shop,Restaurant,Café,Gym,Thai Restaurant,Bakery,Pizza Place,Salad Place,Hotel,Concert Hall
9,"Dufferin, Dovercourt Village",0,Bakery,Pharmacy,Grocery Store,Brewery,Café,Recording Studio,Bar,Supermarket,Bank,Middle Eastern Restaurant


We can see a that Coffe Shop and Café are very popular, let's dig deeper into this analyzing the df columns

In [71]:
cluster1['1st Most Common Venue'].value_counts()

Coffee Shop           14
Café                   5
Park                   2
Greek Restaurant       1
Pizza Place            1
Clothing Store         1
Airport Service        1
Bar                    1
Health Food Store      1
Pub                    1
Light Rail Station     1
Bakery                 1
Grocery Store          1
Sandwich Place         1
Gift Shop              1
Thai Restaurant        1
Name: 1st Most Common Venue, dtype: int64

In [65]:
cluster1['2nd Most Common Venue'].value_counts()

Café                   8
Coffee Shop            5
Hotel                  2
Dessert Shop           2
Restaurant             2
Yoga Studio            1
Sushi Restaurant       1
Trail                  1
Park                   1
Mexican Restaurant     1
Diner                  1
Pharmacy               1
Japanese Restaurant    1
Breakfast Spot         1
Aquarium               1
Airport Lounge         1
Clothing Store         1
Pizza Place            1
Bookstore              1
Cocktail Bar           1
Name: 2nd Most Common Venue, dtype: int64

In [66]:
cluster1['3rd Most Common Venue'].value_counts()

Coffee Shop              4
Café                     4
Restaurant               3
Bar                      2
Italian Restaurant       2
Bakery                   2
Bank                     1
Airport Terminal         1
Cocktail Bar             1
Sandwich Place           1
Breakfast Spot           1
Park                     1
Grocery Store            1
Light Rail Station       1
Pizza Place              1
Pub                      1
Vietnamese Restaurant    1
Gay Bar                  1
Cuban Restaurant         1
Pet Store                1
Sporting Goods Shop      1
Recording Studio         1
Gastropub                1
Name: 3rd Most Common Venue, dtype: int64

In [67]:
cluster1['4th Most Common Venue'].value_counts()

Restaurant                   5
Hotel                        2
Italian Restaurant           2
Sushi Restaurant             2
Japanese Restaurant          2
Sandwich Place               1
Boat or Ferry                1
Mexican Restaurant           1
Movie Theater                1
Pub                          1
Seafood Restaurant           1
Vietnamese Restaurant        1
Café                         1
Thai Restaurant              1
Middle Eastern Restaurant    1
Women's Store                1
Beer Bar                     1
Gym                          1
Grocery Store                1
Brewery                      1
Health & Beauty Service      1
Bakery                       1
History Museum               1
Cheese Shop                  1
Nightclub                    1
Pizza Place                  1
Name: 4th Most Common Venue, dtype: int64

In [68]:
cluster1['5th Most Common Venue'].value_counts()

Japanese Restaurant              3
Café                             3
Italian Restaurant               3
Gastropub                        2
Pub                              2
Bookstore                        2
Grocery Store                    1
Burrito Place                    1
Liquor Store                     1
Sandwich Place                   1
Vegetarian / Vegan Restaurant    1
Department Store                 1
Harbor / Marina                  1
Men's Store                      1
Mexican Restaurant               1
Brewery                          1
American Restaurant              1
Sushi Restaurant                 1
Coffee Shop                      1
Food & Drink Shop                1
Gym                              1
Fast Food Restaurant             1
Boutique                         1
Restaurant                       1
Thai Restaurant                  1
Name: 5th Most Common Venue, dtype: int64

**We can see a huge disproportion between Cafés and Coffee Shops and all other categories.**

**As a conclusion you don't need to worry of running out of coffee in Toronto**